In [4]:
from utils import measure_accuracy, read_jsonl

inference_file = '/Users/ryanarman/code/lab/banking77/notebooks/data/system_prompt_v2_lora_results_2897.jsonl'
inference_data = read_jsonl(inference_file)
accuracy, correct, total, errors, incorrect_list = measure_accuracy(inference_data)

inference_file_corrected = '/Users/ryanarman/code/lab/banking77/notebooks/data/system_prompt_v2_improved_train_results_3015.jsonl'
inference_data = read_jsonl(inference_file_corrected)
accuracy_corrected, correct_corrected, total_corrected, errors_corrected, incorrect_list_corrected = measure_accuracy(inference_data)

Accuracy: 88.60% (2729/3080)
Accuracy: 89.51% (2757/3080)


In [16]:
print(len(incorrect_list),  len(incorrect_list_corrected))

351 323


In [14]:
from collections import Counter

def get_most_common_pairs(incorrect_list):
    wihout_index = [sorted([item[1], item[2]]) for item in incorrect_list]
    wihout_index_str = [f'{item[0]}-{item[1]}' for item in wihout_index]
    cnt = Counter(wihout_index_str)
    most_common_items = cnt.most_common(10)
    most_common_pairs = [item[0] for item in most_common_items]
    return cnt, most_common_pairs

pairs_count, most_common_pairs = get_most_common_pairs(incorrect_list)
pairs_count_corrected, most_common_pairs_corrected = get_most_common_pairs(incorrect_list_corrected)


In [15]:
len(pairs_count), len(pairs_count_corrected)

(182, 169)

In [11]:
most_common_pairs

['33-36',
 '11-12',
 '48-66',
 '25-27',
 '5-67',
 '7-35',
 '56-64',
 '5-66',
 '16-28',
 '16-22']

In [ ]:
['33-36', '11-12', '48-66', '25-27', '5-67', '7-35', '56-64', '5-66', '16-28', '16-22']

In [ ]:
['48-66','66-67', '5-66', '56-65', '7-35', '5-67', '21-38', '51-53', '11-12', '9-24']

In [17]:
most_common_pairs_corrected

['33-36',
 '25-27',
 '11-12',
 '5-66',
 '48-66',
 '16-22',
 '16-28',
 '7-35',
 '56-64',
 '48-67']

In [ ]:
['33-36', '25-27', '11-12', '5-66', '48-66', '16-22', '16-28', '7-35', '56-64', '48-67']


In [13]:
for pair in most_common_pairs:
    pair_split = pair.split('-')
    p1 = (int(pair_split[0]), int(pair_split[1]))
    p2 = (int(pair_split[1]), int(pair_split[0]))

    index_list_of_incorrect_pairs = []
    for item in incorrect_list:
        if item[1:] == p1 or item[1:] == p2:
            # print(item)
            index_list_of_incorrect_pairs.append(item[0])


    print(f"rows with incorrect pairs for {pair}")
    print(index_list_of_incorrect_pairs)
    print("-"*100)

rows with incorrect pairs for 33-36
[247, 250, 252, 255, 264, 265, 268, 271, 276, 410, 428]
----------------------------------------------------------------------------------------------------
rows with incorrect pairs for 11-12
[3, 5, 9, 11, 22, 32, 281, 305, 312]
----------------------------------------------------------------------------------------------------
rows with incorrect pairs for 48-66
[842, 851, 861, 866, 867, 868, 878, 1862, 1867]
----------------------------------------------------------------------------------------------------
rows with incorrect pairs for 25-27
[1721, 1726, 1730, 1739, 1754, 1755, 1756, 1758]
----------------------------------------------------------------------------------------------------
rows with incorrect pairs for 5-67
[2048, 2053, 2065, 2682, 2685, 2691, 2708]
----------------------------------------------------------------------------------------------------
rows with incorrect pairs for 7-35
[2160, 2173, 2176, 2178, 2190, 2196, 2314]
-----

In [21]:
# Analysis of misclassification patterns across validation, corrected test, and original test sets

validation_top10 = ['48-66','66-67', '5-66', '56-65', '7-35', '5-67', '21-38', '51-53', '11-12', '9-24']
corrected_test_top10 = ['33-36', '25-27', '11-12', '5-66', '48-66', '16-22', '16-28', '7-35', '56-64', '48-67']
original_test_top10 = ['33-36', '11-12', '48-66', '25-27', '5-67', '7-35', '56-64', '5-66', '16-28', '16-22']

# Convert to sets for easier comparison
val_set = set(validation_top10)
corrected_set = set(corrected_test_top10)
original_set = set(original_test_top10)

print("="*120)
print("MISCLASSIFICATION PATTERN ANALYSIS")
print("="*120)

print("\n1. PAIRS THAT PERSIST ACROSS ALL THREE SETS (Core Confusion Pairs):")
persistent_pairs = val_set & corrected_set & original_set
print(f"   {sorted(persistent_pairs)}")
print(f"   Count: {len(persistent_pairs)}")
print(f"   These are the most fundamental confusion pairs that appear regardless of dataset or model state.")

print("\n2. PAIRS IN VALIDATION BUT NOT IN TEST SETS (Validation-Specific Issues):")
val_only = val_set - corrected_set - original_set
print(f"   {sorted(val_only)}")
print(f"   Count: {len(val_only)}")
print(f"   These pairs were problematic in validation but may have been addressed or don't appear in test sets.")

print("\n3. PAIRS IN TEST SETS BUT NOT IN VALIDATION (Test-Specific Issues):")
test_only = (corrected_set | original_set) - val_set
print(f"   {sorted(test_only)}")
print(f"   Count: {len(test_only)}")
print(f"   These pairs are problematic in test sets but weren't top issues in validation.")

print("\n4. PAIRS IN CORRECTED TEST BUT NOT ORIGINAL TEST (New Issues After Correction):")
new_after_correction = corrected_set - original_set
print(f"   {sorted(new_after_correction)}")
print(f"   Count: {len(new_after_correction)}")
print(f"   These pairs became more problematic after fine-tuning corrections.")

print("\n5. PAIRS IN ORIGINAL TEST BUT NOT CORRECTED TEST (Fixed Issues):")
fixed_after_correction = original_set - corrected_set
print(f"   {sorted(fixed_after_correction)}")
print(f"   Count: {len(fixed_after_correction)}")
print(f"   These pairs were problematic in original test but improved after correction.")

print("\n6. OVERLAP BETWEEN VALIDATION AND TEST SETS:")
val_test_overlap = (val_set & (corrected_set | original_set))
print(f"   {sorted(val_test_overlap)}")
print(f"   Count: {len(val_test_overlap)}")
print(f"   Percentage of validation pairs that appear in test: {len(val_test_overlap)/len(val_set)*100:.1f}%")

print("\n" + "="*120)
print("KEY INSIGHTS:")
print("="*120)

# Get label names for context
system_message = inference_data[0]['messages'][0]['content'] if len(inference_data) > 0 else ""

def get_label_name(label_id, system_message):
    """Extract label name for a given ID from system message."""
    import re
    pattern = rf'{label_id}:\s*([^\n]+)'
    match = re.search(pattern, system_message)
    if match:
        return match.group(1).strip()
    return f"Unknown_{label_id}"

print("\nA. PERSISTENT CONFUSION PAIRS (appear in all sets):")
for pair in sorted(persistent_pairs):
    id1, id2 = map(int, pair.split('-'))
    name1 = get_label_name(id1, system_message)
    name2 = get_label_name(id2, system_message)
    print(f"   {pair}: {name1} <-> {name2}")

print("\nB. TRANSFER-RELATED CONFUSIONS:")
transfer_pairs = [p for p in (val_set | corrected_set | original_set) 
                  if any('transfer' in get_label_name(int(p.split('-')[i]), system_message).lower() 
                         for i in [0,1])]
print(f"   Found {len(transfer_pairs)} transfer-related confusion pairs")
for pair in sorted(transfer_pairs)[:10]:
    id1, id2 = map(int, pair.split('-'))
    name1 = get_label_name(id1, system_message)
    name2 = get_label_name(id2, system_message)
    print(f"   {pair}: {name1} <-> {name2}")

print("\nC. CARD-RELATED CONFUSIONS:")
card_pairs = [p for p in (val_set | corrected_set | original_set) 
              if any('card' in get_label_name(int(p.split('-')[i]), system_message).lower() 
                     for i in [0,1])]
print(f"   Found {len(card_pairs)} card-related confusion pairs")
for pair in sorted(card_pairs)[:10]:
    id1, id2 = map(int, pair.split('-'))
    name1 = get_label_name(id1, system_message)
    name2 = get_label_name(id2, system_message)
    print(f"   {pair}: {name1} <-> {name2}")

print("\n" + "="*120)
print("INTERPRETATION:")
print("="*120)
persistent_count = len(persistent_pairs)
print(f"""
1. PERSISTENT PAIRS ({persistent_count} pairs): These are the most critical confusion pairs that 
   appear across all datasets. They represent fundamental semantic overlaps that need targeted training.

2. VALIDATION vs TEST DIFFERENCES: The validation set identified some pairs that don't appear as 
   frequently in test sets, suggesting either:
   - Validation set has different distribution
   - Some issues were partially addressed
   - Test set has different challenging pairs

3. CORRECTION IMPACT: Comparing original vs corrected test shows:
   - Some pairs improved (fixed_after_correction: {sorted(fixed_after_correction)})
   - Some pairs got worse (new_after_correction: {sorted(new_after_correction)})
   - This suggests fine-tuning may have over-corrected some issues while creating new ones

4. TRANSFER CONFUSIONS: Many persistent pairs involve transfer-related intents, indicating this 
   is a particularly challenging domain with subtle distinctions.

5. RECOMMENDATION: Focus training data augmentation on persistent pairs, especially transfer-related
   ones, with more diverse examples highlighting the nuanced differences.
""")


MISCLASSIFICATION PATTERN ANALYSIS

1. PAIRS THAT PERSIST ACROSS ALL THREE SETS (Core Confusion Pairs):
   ['11-12', '48-66', '5-66', '7-35']
   Count: 4
   These are the most fundamental confusion pairs that appear regardless of dataset or model state.

2. PAIRS IN VALIDATION BUT NOT IN TEST SETS (Validation-Specific Issues):
   ['21-38', '51-53', '56-65', '66-67', '9-24']
   Count: 5
   These pairs were problematic in validation but may have been addressed or don't appear in test sets.

3. PAIRS IN TEST SETS BUT NOT IN VALIDATION (Test-Specific Issues):
   ['16-22', '16-28', '25-27', '33-36', '48-67', '56-64']
   Count: 6
   These pairs are problematic in test sets but weren't top issues in validation.

4. PAIRS IN CORRECTED TEST BUT NOT ORIGINAL TEST (New Issues After Correction):
   ['48-67']
   Count: 1
   These pairs became more problematic after fine-tuning corrections.

5. PAIRS IN ORIGINAL TEST BUT NOT CORRECTED TEST (Fixed Issues):
   ['5-67']
   Count: 1
   These pairs were 